In [ ]:
spark.sparkContext._jsc.hadoopConfiguration().get("fs.s3a.endpoint")


In [ ]:

hadoop_conf

In [ ]:
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3a://raw/nubank/Nubank_2025-06-10.csv")

df_csv.show()

In [ ]:
# Use Python, não %%sql
spark.sql("""
    CREATE OR REPLACE TABLE demo.raw.nubank_faturas
    USING csv
    OPTIONS (
        path 's3://raw/nubank/',
        header 'true',
        inferSchema 'true'
    )
""")

In [ ]:
from pyspark.sql.functions import *

# 1️⃣ Ler CSV (via tabela externa Iceberg)
spark.sql("""
    CREATE OR REPLACE TABLE demo.raw.nubank_faturas
    USING csv
    OPTIONS (
        path 's3://raw/nubank/',
        header 'true',
        inferSchema 'true'
    )
""")

df_raw = spark.table("demo.raw.nubank_faturas")
print(f"📄 Total de registros RAW: {df_raw.count()}")

# 2️⃣ Transformar
df_silver = df_raw \
    .filter(col("_c0").isNotNull()) \
    .withColumn("processed_at", current_timestamp())

# 3️⃣ Salvar no Silver (agora como tabela Iceberg nativa)
df_silver.write \
    .format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("demo.silver.nubank_faturas")

print("✅ Pipeline concluído!")

# 4️⃣ Consultar no Silver
spark.sql("SELECT * FROM demo.silver.nubank_faturas LIMIT 10").show()